<a href="https://colab.research.google.com/github/jhchang/tnr-ML/blob/main/Cat_individual_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import files
# files.upload()  # Select kaggle.json here

In [2]:
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json  # Set permissions

In [3]:
import gc
gc.collect()

112

In [4]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [5]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# !wget -O /content/drive/MyDrive/datasets/cat-individuals/cat-individuals.zip https://www.kaggle.com/api/v1/datasets/download/timost1234/cat-individuals

# WARNING!!! CHANGE zip_path ACCORDING TO WHETHER OR NOT YOU HAVE MADE A ZIP FILE THAT HAS ALL THE PHOTOS CLEANED AND CONVERTED TO PNG OF JPG

In [8]:
import zipfile
from tqdm import tqdm
import os

# if you are downlading from the og dataset, you have to use
# zip_path = "/content/drive/MyDrive/datasets/cat-individuals/cat-individuals.zip"
# and then convert all the heic pictures to jpg and then zip it up. otherwise use below
zip_path = "/content/drive/MyDrive/datasets/cat-individuals/cat-individuals-converted.zip"
extract_to = "/content/cat-individuals/"

# Count the total number of files in the zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    total_files = len(zip_ref.infolist())

    print(f"📦 Unzipping {total_files} files to {extract_to}...")

    with tqdm(total=total_files, unit="files") as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(member=file, path=extract_to)
            pbar.update(1)

print("✅ Unzip complete!")


📦 Unzipping 13825 files to /content/cat-individuals/...


100%|██████████| 13825/13825 [03:29<00:00, 65.88files/s]

✅ Unzip complete!


In [9]:
# original zip file extracts to a folder called cat_individuals_dataset
# dataset_dir = "/content/cat-individuals/cat_individuals_dataset"
dataset_dir = "/content/cat-individuals"


In [10]:
import os

file_count = sum(len(files) for _, _, files in os.walk(dataset_dir))

print(f"Total number of files: {file_count}")


Total number of files: 13825


In [11]:
import os

folder_file_counts = {}

for root, dirs, files in os.walk(dataset_dir):
    folder_file_counts[root] = len(files)

# Display nicely
for folder, count in folder_file_counts.items():
    print(f'{folder}: {count} files')


/content/cat-individuals: 0 files
/content/cat-individuals/0471: 11 files
/content/cat-individuals/0372: 12 files
/content/cat-individuals/0376: 10 files
/content/cat-individuals/0363: 11 files
/content/cat-individuals/0253: 33 files
/content/cat-individuals/0364: 9 files
/content/cat-individuals/0369: 12 files
/content/cat-individuals/0163: 68 files
/content/cat-individuals/0378: 16 files
/content/cat-individuals/0337: 17 files
/content/cat-individuals/0071: 40 files
/content/cat-individuals/0436: 21 files
/content/cat-individuals/0296: 21 files
/content/cat-individuals/0250: 10 files
/content/cat-individuals/0027: 51 files
/content/cat-individuals/0424: 20 files
/content/cat-individuals/0171: 16 files
/content/cat-individuals/0497: 37 files
/content/cat-individuals/0415: 21 files
/content/cat-individuals/0007: 41 files
/content/cat-individuals/0481: 33 files
/content/cat-individuals/0218: 12 files
/content/cat-individuals/0493: 27 files
/content/cat-individuals/0488: 49 files
/conten

In [12]:
import os
from pathlib import Path

DATASET_DIR = dataset_dir  # Change this to your dataset folder

VALID_EXTENSIONS = {".png", ".jpg", ".jpeg"}

non_image_files = []

for root, dirs, files in os.walk(DATASET_DIR):
    for file in files:
        ext = Path(file).suffix.lower()
        if ext not in VALID_EXTENSIONS:
            full_path = os.path.join(root, file)
            non_image_files.append(full_path)

if non_image_files:
    print("Found files with unsupported extensions:")
    for f in non_image_files:
        print(f)
else:
    print("All files have valid extensions (.png, .jpg, .jpeg).")


All files have valid extensions (.png, .jpg, .jpeg).


# The next cells are used to convert the original dataset into all png and jpg compatible files and then zip them back up for faster successive runs

In [13]:
# pip install pillow pyheif

In [14]:
# !pip install pillow-heif

In [15]:
# from pathlib import Path
# import shutil
# from tqdm import tqdm
# from PIL import Image
# import pillow_heif

# pillow_heif.register_heif_opener()

# DATASET_DIR = Path(dataset_dir)  # Update path as needed

# valid_exts = {".jpg", ".jpeg", ".png"}

# for class_folder in DATASET_DIR.iterdir():
#     if not class_folder.is_dir():
#         continue

#     all_files = list(class_folder.rglob("*.*"))
#     print(f"\nProcessing class folder: {class_folder.name} ({len(all_files)} files)")

#     for file_path in tqdm(all_files):
#         if file_path.is_dir():
#             continue

#         ext = file_path.suffix.lower()
#         parent = file_path.parent

#         # Convert HEIC to JPG
#         if ext == ".heic":
#             try:
#                 with Image.open(file_path) as img:
#                     rgb_img = img.convert("RGB")
#                     jpg_path = file_path.with_suffix(".jpg")

#                     # Avoid overwriting if jpg_path exists
#                     count = 1
#                     while jpg_path.exists():
#                         jpg_path = jpg_path.with_name(f"{jpg_path.stem}_{count}.jpg")
#                         count += 1

#                     rgb_img.save(jpg_path, "JPEG")
#                 file_path.unlink()  # delete original heic
#                 print(f"Converted HEIC: {file_path.name} → {jpg_path.name}")

#                 # Move converted jpg to class folder root if needed
#                 if parent != class_folder:
#                     target_path = class_folder / jpg_path.name
#                     if target_path.exists():
#                         stem = target_path.stem
#                         suffix = target_path.suffix
#                         c = 1
#                         while target_path.exists():
#                             target_path = class_folder / f"{stem}_{c}{suffix}"
#                             c += 1
#                     shutil.move(str(jpg_path), str(target_path))

#             except Exception as e:
#                 print(f"Failed to convert {file_path}: {e}")
#             continue

#         # For valid images, move them up if nested
#         elif ext in valid_exts:
#             if parent != class_folder:
#                 target_path = class_folder / file_path.name
#                 if target_path.exists():
#                     stem = target_path.stem
#                     suffix = target_path.suffix
#                     c = 1
#                     while target_path.exists():
#                         target_path = class_folder / f"{stem}_{c}{suffix}"
#                         c += 1
#                 shutil.move(str(file_path), str(target_path))

#         # Remove unsupported files
#         else:
#             try:
#                 file_path.unlink()
#             except Exception as e:
#                 print(f"Failed to delete {file_path}: {e}")


In [16]:
from pathlib import Path

DATASET_DIR = Path(dataset_dir)  # adjust path as needed

# Find nested subfolders (i.e., any folder inside class folders)
nested_dirs = [
    p for p in DATASET_DIR.rglob("*")
    if p.is_dir() and p.parent != DATASET_DIR
]

if nested_dirs:
    print("❌ Found nested subfolders:")
    for d in nested_dirs:
        print("-", d)
else:
    print("✅ No nested subfolders found.")


✅ No nested subfolders found.


In [17]:
from pathlib import Path
from collections import Counter

def count_file_types(folder_path):
    folder = Path(folder_path)
    all_files = folder.rglob("*.*")  # all files with extensions
    ext_counter = Counter()

    for f in all_files:
        ext = f.suffix.lower()
        ext_counter[ext] += 1

    print(f"File type counts in '{folder}':")
    for ext, count in ext_counter.most_common():
        print(f"{ext}: {count}")

# Usage example:
count_file_types(dataset_dir)


File type counts in '/content/cat-individuals':
.jpg: 13761
.jpeg: 62
.png: 2


In [18]:
# import os
# import zipfile

# # Source folder — where your cleaned dataset is
# SOURCE_DIR = "/content/cat-individuals/cat_individuals_dataset"  # or wherever your files are

# # Destination zip file path on Google Drive
# DEST_ZIP = "/content/drive/MyDrive/datasets/cat-individuals/cat-individuals-converted.zip"

In [19]:
# import os
# import zipfile
# from tqdm import tqdm

# def zip_folder_with_progress(source_dir, output_zip):
#     # Collect all file paths
#     file_paths = []
#     for root, _, files in os.walk(source_dir):
#         for file in files:
#             file_path = os.path.join(root, file)
#             arcname = os.path.relpath(file_path, start=source_dir)
#             file_paths.append((file_path, arcname))

#     # Zip with progress bar
#     with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for file_path, arcname in tqdm(file_paths, desc="Zipping files", unit="file"):
#             zipf.write(file_path, arcname)

#     print(f"✅ Zipped folder saved to: {output_zip}")


In [20]:
# zip_folder_with_progress(SOURCE_DIR, DEST_ZIP)


# End of HEIC conversion and zipping
---



In [21]:
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

# Configuration
DATASET_DIR = dataset_dir  # Root directory with subfolders for each cat
VALID_EXTENSIONS = {".png", ".jpg", ".jpeg"}
IMG_SIZE = (224, 224)
BATCH_SIZE = 2
SEED = 123

# Step 1: Gather all image paths and labels
image_paths = []
labels = []
class_names = sorted([d.name for d in Path(DATASET_DIR).iterdir() if d.is_dir()])
class_to_index = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_dir = Path(DATASET_DIR) / class_name
    for img_path in class_dir.glob("*"):
        if img_path.suffix.lower() in VALID_EXTENSIONS:
            image_paths.append(str(img_path))
            labels.append(class_to_index[class_name])

image_paths = np.array(image_paths)
labels = np.array(labels)


# Step 2: Split using sklearn
X_train, X_temp, y_train, y_temp = train_test_split(
    image_paths, labels, test_size=0.3, random_state=SEED, stratify=labels
)
print(X_train.shape, y_train.shape)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, stratify=y_temp
)
print(X_val.shape, X_test.shape, y_val.shape, y_test.shape)

# Step 3: Helper function to create datasets
def preprocess_image(path, label):
    img_raw = tf.io.read_file(path)
    if tf.strings.regex_full_match(path, ".*\\.png"):
        img = tf.image.decode_png(img_raw, channels=3)
    else:
        img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return img, label



def create_dataset(paths, labels, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Step 4: Create datasets
train_ds = create_dataset(X_train, y_train)
val_ds = create_dataset(X_val, y_val)
test_ds = create_dataset(X_test, y_test, shuffle=False)

print("✅ Datasets ready:")
print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")



(9677,) (9677,)
(2074,) (2074,) (2074,) (2074,)
✅ Datasets ready:
Train: 9677 | Val: 2074 | Test: 2074


In [22]:
from pathlib import Path

zero_byte_files = [p for p in image_paths if Path(p).stat().st_size == 0]
print("Zero byte files:", zero_byte_files)


Zero byte files: []


In [23]:
print("Train batches:", len(train_ds))
print("Val batches:", len(val_ds))


Train batches: 4839
Val batches: 1037


In [24]:
from PIL import Image
from pathlib import Path
from tqdm import tqdm

valid_exts = [".jpg", ".jpeg", ".png"]
valid_image_paths = []
corrupt_files = []

for class_name in class_names:
    class_dir = Path(DATASET_DIR) / class_name
    for img_path in tqdm(list(class_dir.glob("*"))):
        if img_path.suffix.lower() in valid_exts:
            try:
                with Image.open(img_path) as img:
                    img.verify()  # Does not load the image fully
                valid_image_paths.append(str(img_path))
            except Exception as e:
                print(f"❌ Corrupted: {img_path} -> {e}")
                corrupt_files.append(str(img_path))

print(f"✅ Valid images: {len(valid_image_paths)}")
print(f"❌ Corrupt images: {len(corrupt_files)}")

100%|██████████| 27/27 [00:00<00:00, 550.90it/s]

✅ Valid images: 13825
❌ Corrupt images: 0


In [26]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Model configuration
IMG_SHAPE = (224, 224, 3)
NUM_CLASSES = len(class_names)  # from earlier label loading
LEARNING_RATE = 1e-4
EPOCHS = 25

# Load base model
base_model = EfficientNetB0(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze base model

# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

In [ ]:
# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/25
4839/4839 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.0460 - loss: 6.0902

4839/4839 ━━━━━━━━━━━━━━━━━━━━ 1849s 253ms/step - accuracy: 0.0460 - loss: 6.0902 - val_accuracy: 0.0501 - val_loss: 5.9441
Epoch 2/25
4839/4839 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.0544 - loss: 5.9431

4839/4839 ━━━━━━━━━━━━━━━━━━━━ 1817s 240ms/step - accuracy: 0.0544 - loss: 5.9431 - val_accuracy: 0.0501 - val_loss: 5.9306
Epoch 3/25
4839/4839 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.0501 - loss: 5.9516

4839/4839 ━━━━━━━━━━━━━━━━━━━━ 1776s 249ms/step - accuracy: 0.0501 - loss: 5.9516 - val_accuracy: 0.0501 - val_loss: 5.9194
Epoch 4/25
4839/4839 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.0504 - loss: 5.9594

4839/4839 ━━━━━━━━━━━━━━━━━━━━ 1753s 237ms/step - accuracy: 0.0504 - loss: 5.9594 - val_accuracy: 0.0501 - val_loss: 5.9123
Epoch 5/25
4839/4839 ━━━━━━━━━━━━━━━━━━━━ 1916s 263ms/step - accuracy: 0.0537 - loss: 5.9308 - val_accuracy: 0.0501 - val_loss: 5.9160
Epoch 6/25
1324/4839 ━━━━━━━━━━━━━━━━━━━━ 10:54 186ms/step - accuracy: 0.0587 - loss: 5.9149